LLM takes in user data & recommends "sessions" to the users -> pass into vector database to fetch real sessions. We first initialise our API Keys.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

Then, we will initialise our vector DB client, the first query will take about 20 seconds as there will be authentication, subsequent queries will be much shorter.

In [75]:
from qdrant_client import QdrantClient

QDRANT_URL=os.getenv("QDRANT_URL")
QDRANT_API_KEY=os.getenv("QDRANT_API_KEY")

qdrant_client = QdrantClient(
	url=QDRANT_URL, 
	api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='star_charts'), CollectionDescription(name='midjourney')]


Now, we create a method to embed the artist's profile and a fetch method to fetch the data.

In [76]:
from openai import OpenAI
openai_client = OpenAI()
openai_client.api_key=os.getenv("OPENAI_API_KEY")

In [97]:
def text_to_embedding(text):
    text = text.replace("\n", " ")
    embeddings = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text,
        encoding_format="float"
    )
    return embeddings

def fetch_recommended_sessions(artist_profile, collection_name=os.getenv("COLLECTION_NAME"), limit=5):
    artist_embedding = text_to_embedding(artist_profile)
    similar_sessions = qdrant_client.search(
        collection_name=collection_name,
        query_vector=artist_embedding,
        limit=limit
    )
    return [session.payload["text"] for session in similar_sessions]

Now lets test the accuracy of these queries.

In [ ]:
query_embedding = text_to_embedding("")